In [1]:
!rm -rf ./logs/ 
%run Model.ipynb
%run dataset_loader.ipynb
import tensorflow as tf
#from dataset_loader import load_dataset

#tf.enable_eager_execution()
INPUT_WIDTH = 64
INPUT_HEIGHT = 65

INPUT_SIZE = 64

INPUT_CHANNELS = 3

NUM_CLASSES = 5

LEARNING_RATE = 0.001   # Original value: 0.01
MOMENTUM = 0.9
KEEP_PROB = 1

EPOCHS = 500
BATCH_SIZE = 128

train_sub_dir = '/data/image_format_small_dataset/*/*'
val_sub_dir = '/data/image_format_small_dataset_val/*/*'
test_sub_dir = '/data/image_format_small_dataset_test/*/*'

frequency_train_sub_dir = '/data/cogisen_frequency_data/image_format_small_dataset_cogisen_updated/*/*/*.csv'
frequency_val_sub_dir = '/data/image_format_small_dataset_val_cogisen_updated/*/*/*.csv'
frequency_test_sub_dir = '/data/image_format_small_dataset_test_cogisen_updated/*/*/*.csv'


home_dir = os.path.dirname(os.path.realpath('__file__'))

train_dir = home_dir + train_sub_dir
val_dir = home_dir + val_sub_dir
test_dir = home_dir + test_sub_dir


frequency_train_dir = home_dir + frequency_train_sub_dir
frequency_val_dir = home_dir + frequency_val_sub_dir
frequency_test_dir = home_dir + frequency_test_sub_dir



training_dataset= load_dataset(img_dir= train_dir,frequency_dir=frequency_train_dir,minibatch_size=BATCH_SIZE,image_size=INPUT_SIZE,grayscale=True)


val_dataset = load_dataset(img_dir= val_dir,frequency_dir=frequency_train_dir,minibatch_size=BATCH_SIZE,image_size=INPUT_SIZE,grayscale=True)

test_dataset = load_dataset(img_dir= test_dir,frequency_dir=frequency_train_dir,minibatch_size=BATCH_SIZE,image_size=INPUT_SIZE,grayscale=True)

experiment_no = 2
#logdir = './log/alexnet/FULL_DATA'+str(experiment_no)
logdir = './log/frequency_data/grayscale'





/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/home/shubham/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorboard/compat

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [2]:

handle = tf.placeholder(tf.string, shape=[])


training_iterator = tf.data.Iterator.from_structure(training_dataset.output_types,training_dataset.output_shapes)
training_init_op = training_iterator.make_initializer(training_dataset)




Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_classes(iterator)`.


In [3]:
iterator = tf.data.Iterator.from_string_handle(
    handle, training_iterator.output_types,training_dataset.output_shapes)

next_training_element = iterator.get_next()

In [4]:
with tf.device('/gpu:0'):
    alexnet = AlexNet(next_training_element,input_height=INPUT_HEIGHT,input_width=INPUT_WIDTH, num_classes=NUM_CLASSES)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Y shape:  (?, 5)
layer8_logits:  (?, 5)


In [5]:
with tf.device('/gpu:0'):

    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
        print('Training dataset...')
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        file_writer = tf.summary.FileWriter(logdir=logdir, graph=sess.graph)

        training_string_handle =  sess.run(training_iterator.string_handle())

        summary_operation = tf.summary.merge_all()

        for epoch in range(EPOCHS):

            sess.run(training_init_op,feed_dict={handle:training_string_handle})
            _, accuracy,summary,Y,final_logits= sess.run([alexnet.training_operation,alexnet.accuracy_operation,summary_operation,alexnet.Y,alexnet.final_logits],feed_dict={alexnet.dropout_keep_prob: alexnet.keep_prob,handle:training_string_handle})


            file_writer.add_summary(summary, epoch)
            print("accuracy for epoch {0}  :  {1}".format(epoch,accuracy))

            #print("Y: ",Y)
            #print("logits: ",final_logits)
            
            

Training dataset...
accuracy for epoch 0  :  0.171875
accuracy for epoch 1  :  0.1796875
accuracy for epoch 2  :  0.1328125
accuracy for epoch 3  :  0.2265625
accuracy for epoch 4  :  0.234375
accuracy for epoch 5  :  0.1953125
accuracy for epoch 6  :  0.2109375
accuracy for epoch 7  :  0.25
accuracy for epoch 8  :  0.2421875
accuracy for epoch 9  :  0.1484375
accuracy for epoch 10  :  0.2265625
accuracy for epoch 11  :  0.109375
accuracy for epoch 12  :  0.15625
accuracy for epoch 13  :  0.21875
accuracy for epoch 14  :  0.234375
accuracy for epoch 15  :  0.15625
accuracy for epoch 16  :  0.1796875
accuracy for epoch 17  :  0.2109375
accuracy for epoch 18  :  0.1796875
accuracy for epoch 19  :  0.2578125
accuracy for epoch 20  :  0.2734375
accuracy for epoch 21  :  0.21875
accuracy for epoch 22  :  0.203125
accuracy for epoch 23  :  0.1640625
accuracy for epoch 24  :  0.21875
accuracy for epoch 25  :  0.1484375
accuracy for epoch 26  :  0.265625
accuracy for epoch 27  :  0.2109375
acc

accuracy for epoch 230  :  0.1875
accuracy for epoch 231  :  0.21875
accuracy for epoch 232  :  0.1796875
accuracy for epoch 233  :  0.2265625
accuracy for epoch 234  :  0.140625
accuracy for epoch 235  :  0.2109375
accuracy for epoch 236  :  0.234375
accuracy for epoch 237  :  0.1796875
accuracy for epoch 238  :  0.1875
accuracy for epoch 239  :  0.1875
accuracy for epoch 240  :  0.2421875
accuracy for epoch 241  :  0.1875
accuracy for epoch 242  :  0.21875
accuracy for epoch 243  :  0.1875
accuracy for epoch 244  :  0.203125
accuracy for epoch 245  :  0.21875
accuracy for epoch 246  :  0.2421875
accuracy for epoch 247  :  0.125
accuracy for epoch 248  :  0.15625
accuracy for epoch 249  :  0.2109375
accuracy for epoch 250  :  0.203125
accuracy for epoch 251  :  0.15625
accuracy for epoch 252  :  0.1796875
accuracy for epoch 253  :  0.21875
accuracy for epoch 254  :  0.2109375
accuracy for epoch 255  :  0.1484375
accuracy for epoch 256  :  0.234375
accuracy for epoch 257  :  0.1640625


accuracy for epoch 458  :  0.1875
accuracy for epoch 459  :  0.234375
accuracy for epoch 460  :  0.1953125
accuracy for epoch 461  :  0.15625
accuracy for epoch 462  :  0.2109375
accuracy for epoch 463  :  0.1796875
accuracy for epoch 464  :  0.2265625
accuracy for epoch 465  :  0.1953125
accuracy for epoch 466  :  0.2578125
accuracy for epoch 467  :  0.1875
accuracy for epoch 468  :  0.1953125
accuracy for epoch 469  :  0.21875
accuracy for epoch 470  :  0.15625
accuracy for epoch 471  :  0.265625
accuracy for epoch 472  :  0.2421875
accuracy for epoch 473  :  0.2578125
accuracy for epoch 474  :  0.1875
accuracy for epoch 475  :  0.25
accuracy for epoch 476  :  0.234375
accuracy for epoch 477  :  0.1640625
accuracy for epoch 478  :  0.1875
accuracy for epoch 479  :  0.2265625
accuracy for epoch 480  :  0.1796875
accuracy for epoch 481  :  0.171875
accuracy for epoch 482  :  0.25
accuracy for epoch 483  :  0.203125
accuracy for epoch 484  :  0.21875
accuracy for epoch 485  :  0.2265625

In [6]:

val_iterator = tf.data.Iterator.from_structure(val_dataset.output_types,val_dataset.output_shapes)
val_init_op = val_iterator.make_initializer(val_dataset)



In [7]:
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    print('validation dataset...')
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(val_init_op)
    val_string_handle =  sess.run(val_iterator.string_handle())
    val_accuracy = sess.run([alexnet.accuracy_operation],feed_dict={alexnet.dropout_keep_prob: alexnet.keep_prob, 
                                                                     handle:val_string_handle})
    

    print("accuracy for validation set :  {0}".format(val_accuracy))



validation dataset...
accuracy for validation set :  [0.171875]


In [8]:

test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,test_dataset.output_shapes)
test_init_op = test_iterator.make_initializer(test_dataset)



In [9]:
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    print('Testing dataset...')
    
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    sess.run(test_init_op)
    test_string_handle =  sess.run(test_iterator.string_handle())
    test_accuracy = sess.run([alexnet.accuracy_operation],feed_dict={alexnet.dropout_keep_prob: alexnet.keep_prob, 
                                                                     handle:test_string_handle})
    

    print("accuracy for test set :  {0}".format(test_accuracy))



Testing dataset...
accuracy for test set :  [0.1953125]
